# Bi-Directional LSTM

In [1]:
# Imports 
import torch
import numpy as np
import tqdm

import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyperparameters setting
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.005
batch_size = 64
num_epochs = 5


class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        
        return out

# Load the Data Train & Test
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
        
# Initialize the Network
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the Network
for epoch in tqdm.tqdm(range(num_epochs)):
    for batch_idx, (data, targets) in enumerate(train_loader):
        
        # Convert the data to the CUDA for faster training
        data = data.to(device=device).squeeze(1) # bcoz MNIST has 1x28x28 but RNN expects N x 28 x 28 ---.squeeze(1) will remove 1 from that particular axis
        targets = targets.to(device=device)
        
        # Flattening the data
#         data = data.reshape(data.shape[0], -1)
        
        # Forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # Backward
        optimizer.zero_grad() # so that they dont store the gradrients
        loss.backward() # gradients computed here
        
        # gradient steps 
        optimizer.step() # here we update the weights based on the gradients computed on top loss.backward()
        
        
# Check Accuracy on training & test set 

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on Train Data!!... ")
    else:
        print(" Checking accuracy on Test Data!!  ")
    
    num_correct = 0
    num_samples = 0
    model.eval() # let the model know that it is in evaluation mode
    
    # to let the model know that you dont have to compute the gradients while doing the tesing/accuracy checking
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)
            
             # x = x.reshape(x.shape[0], -1) # Flattening
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        # print('{}/{} with accuracy {}'.format(num_correct, num_samples, (num_correct/num_samples)*100, 2))
        
    model.train()
    return num_correct/num_samples
    

print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")        



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:40<00:00, 20.19s/it]


Checking accuracy on Train Data!!... 
Accuracy on training set: 98.581665
 Checking accuracy on Test Data!!  
Accuracy on test set: 98.12
